<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.1
    Uninstalling protobuf-6.31.1:
      Successfully uninstalled protobuf-6.31.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-07-08 10:43:56
-------------------
qualified stocks: 96
with latest results: 31
still star stocks: 20
-------------------
Initial Investment:  1.17 C
CY Investment:  1.44 C
Reserve:  9.00 K
Current:  1.37 C
-------------------
Today PnL: -16.82 K (-0.12%)
Current PnL: -13.32 L (-9.28%)
CY Booked + Current PnL: -6.36 L (-4.43%)
-------------------
Total profit:  4.73 L
Total loss:  -18.05 L
-------------------
Total Booked + Current PnL: 20.62 L (17.69%)
Total Booked PnL: 33.94 L (29.12%)
Curr Year Booked PnL: 6.96 L (5.09%)
Prev Year Booked PnL: 26.98 L (23.15%)
Est FTT:  2.09 C
Est FTT PnL: 71.84 L (52.51%)
Deployed:  1.17 C
Current:  1.37 C
CAGR/XIRR %: 10.81%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'RSP', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
4,APOLLOHOSP,6904.43,8285.00,-16.19,H-LC,91.67,114158.0,10592.0,10114.0,-0.07,10.23,8.86,20.00,32.0,1.05,0.84,25.74,X40N,BTT,HEALTHCARE
39,HINDALCO,651.95,761.55,-19.90,H-LC,85.42,110288.0,5976.0,11558.0,-0.40,5.73,10.48,16.81,27.0,0.52,0.81,22.69,X5K,ATH,METALS
88,UNIONBANK,123.87,163.00,-1.39,M-LC,27.08,172472.0,31632.0,12866.0,-0.32,22.46,7.46,31.59,88.0,2.46,1.26,49.82,XY24,NTT,BANKS
92,VIPIND,488.80,489.00,-212.26,H-SC,97.92,80964.0,-14352.0,14387.0,0.10,-15.06,17.77,0.04,157.0,-1.00,0.59,59.69,OX40N,NTT,MISC
71,SBIN,760.30,863.00,-11.51,M-LC,30.21,213934.0,12454.0,14761.0,0.06,6.18,6.90,13.51,77.0,0.84,1.56,19.56,XY25,NTT,BANKS


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BANKINDIA,116.91,190.00,-31.29,M-MC,8.33,180330.0,522.0,111895.0,-0.40,0.29,62.05,62.52,173.0,0.00,1.32,34.48,XR,NTT,BANKS
38,HEROMOTOCO,4311.81,6039.03,-7.14,H-MC,29.17,149418.0,-1495.0,61949.0,-0.88,-0.99,41.46,40.06,103.0,-0.02,1.09,22.22,AR,ATH,AUTO
60,NATIONALUM,189.63,247.44,-50.15,H-MC,51.04,99984.0,-520.0,31155.0,0.05,-0.52,31.16,30.49,97.0,-0.02,0.73,31.72,MH,ATH,METALS


In [11]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
16,BLUSPRING,226.45,85.1,NaN,NaN,48.96,18637.0,-30956.0,0.0,4.20,-62.42,0.00,-62.42,270.0,-inf,0.14,24.98,XY24,ATH,MISC
32,ENRIN,1377.95,3087.3,NaN,NaN,95.83,77182.0,42733.0,0.0,2.91,124.05,0.00,124.05,274.0,inf,0.56,18.03,AR,ATH,ELECTRICAL
13,BATAINDIA,1550.24,2096.0,-30.44,M-SC,66.67,103999.0,-24671.0,69971.0,2.01,-19.17,67.28,35.20,205.0,-0.35,0.76,6.11,X40,NTT,FOOTWEAR
52,JPPOWER,18.73,26.2,-13.49,L-SC,100.00,174179.0,32337.0,24228.0,1.55,22.80,13.91,39.88,262.0,1.33,1.27,76.52,XY24,NTT,POWER
89,VAIBHAVGBL,350.21,521.0,55.37,X-SC,13.54,103657.0,-48334.0,122460.0,1.31,-31.80,118.14,48.77,62.0,-0.39,0.76,19.82,XR,NTT,JEWELLERY


In [12]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
24,COFFEEDAY,59.14,80.00,-57.75,L-SC,39.58,63110.0,-50439.0,90487.0,-3.55,-44.42,143.38,35.27,265.0,-0.56,0.46,54.46,XR,NTT,HOTELS
65,RELAXO,902.64,1176.00,-30.47,H-SC,92.71,62880.0,-52658.0,87648.0,-2.98,-45.58,139.39,30.28,131.0,-0.60,0.46,22.84,X40N,NTT,FOOTWEAR
3,ALKYLAMINE,2347.98,4546.37,8.78,X-SC,98.96,101183.0,220.0,94313.0,-1.87,0.22,93.21,93.63,67.0,0.00,0.74,50.71,SR,ATH,CHEMICALS
27,DEN,52.79,75.00,120.21,L-SC,80.21,61984.0,-22480.0,58017.0,-1.53,-26.61,93.60,42.07,261.0,-0.39,0.45,31.63,AR,NTT,ENTERTAINMENT
22,CIPLA,1495.00,1795.00,-22.50,H-LC,42.71,99676.0,-489.0,20593.0,-1.51,-0.49,20.66,20.07,30.0,-0.02,0.73,8.37,X40N,BTT,PHARMA


In [13]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
87,UJJIVANSFB,52.77,53.0,45.86,M-SC,23.96,126954.0,-15525.0,16149.0,0.04,-10.90,12.72,0.44,247.0,-0.96,0.93,49.84,OX40N,NTT,BANKS
79,SYMPHONY,1306.42,1306.0,-29.37,M-SC,26.04,150650.0,-20491.0,20443.0,-0.13,-11.97,13.57,-0.03,191.0,-1.00,1.10,8.28,OX40N,NTT,DURABLES
46,INDIGOPNTS,1407.73,1408.0,127.12,H-SC,94.79,148961.0,-25598.0,25636.0,0.04,-14.66,17.21,0.02,133.0,-1.00,1.09,29.17,OX40N,NTT,PAINTS
92,VIPIND,488.80,489.0,-212.26,H-SC,97.92,80964.0,-14352.0,14387.0,0.10,-15.06,17.77,0.04,157.0,-1.00,0.59,59.69,OX40N,NTT,MISC
53,KANSAINER,299.63,340.0,-67.65,H-SC,44.79,225270.0,-44397.0,80737.0,0.22,-16.46,35.84,13.47,137.0,-0.55,1.65,12.75,XY24,NTT,PAINTS


In [14]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,TCS,3794.03,4998.00,-18.70,X-LC,38.54,264974.0,-30960.0,124882.0,-0.43,-10.46,47.13,31.73,1.0,-0.25,1.94,6.06,X40,BTT,IT
48,INFY,1461.46,2275.00,-9.19,X-LC,73.44,290857.0,29256.0,116372.0,-0.13,11.18,40.01,55.67,2.0,0.25,2.13,17.97,X40,BTT,IT
40,HINDUNILVR,2413.81,2723.00,-14.11,X-LC,58.33,269270.0,-1077.0,35705.0,-0.26,-0.40,13.26,12.81,5.0,-0.03,1.97,12.61,XY25,NTT,FMCG
17,BRITANNIA,4983.33,6446.05,17.95,X-LC,71.88,270296.0,41063.0,26219.0,-0.14,17.91,9.70,29.35,10.0,1.57,1.98,28.43,XY25,ATH,FMCG
74,SIEMENS,4406.90,7969.85,-1.77,X-LC,28.12,82950.0,-27222.0,116296.0,1.21,-24.71,140.20,80.85,11.0,-0.23,0.61,22.17,AR,ATH,ELECTRICAL


In [15]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
32,ENRIN,1377.95,3087.30,NaN,NaN,95.83,77182.0,42733.0,0.0,2.91,124.05,0.00,124.05,274.0,inf,0.56,18.03,AR,ATH,ELECTRICAL
28,DIGITIDE,188.38,237.72,NaN,NaN,33.33,50348.0,9093.0,1712.0,0.61,22.04,3.40,26.19,272.0,5.31,0.37,18.54,XY24,ATH,IT
52,JPPOWER,18.73,26.20,-13.49,L-SC,100.00,174179.0,32337.0,24228.0,1.55,22.80,13.91,39.88,262.0,1.33,1.27,76.52,XY24,NTT,POWER
88,UNIONBANK,123.87,163.00,-1.39,M-LC,27.08,172472.0,31632.0,12866.0,-0.32,22.46,7.46,31.59,88.0,2.46,1.26,49.82,XY24,NTT,BANKS


In [16]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
68,SAIL,130.64,228.00,93.57,M-MC,50.00,232057.0,7095.0,160560.0,0.97,3.15,69.19,74.53,189.0,0.04,1.70,34.73,XY24,BTT,STEEL
54,KPIGREEN,497.21,731.64,25.10,H-SC,32.29,128029.0,2732.0,56346.0,-0.95,2.18,44.01,47.15,143.0,0.05,0.94,60.17,MH,ATH,POWER
20,CAMS,3643.00,5250.99,2.76,H-SC,22.92,115528.0,13524.0,31504.0,-0.39,13.26,27.27,44.14,126.0,0.43,0.85,31.65,X40N,ATH,MISC
88,UNIONBANK,123.87,163.00,-1.39,M-LC,27.08,172472.0,31632.0,12866.0,-0.32,22.46,7.46,31.59,88.0,2.46,1.26,49.82,XY24,NTT,BANKS
95,WIPRO,243.46,420.00,-2.86,M-LC,89.58,166005.0,15060.0,94390.0,0.00,9.98,56.86,72.51,71.0,0.16,1.21,14.35,XR,NTT,IT


In [17]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
32,ENRIN,1377.95,3087.30,NaN,NaN,95.83,77182.0,42733.0,0.0,2.91,124.05,0.00,124.05,274.0,inf,0.56,18.03,AR,ATH,ELECTRICAL
94,WHIRLPOOL,1167.49,2270.00,-39.19,M-SC,53.12,200714.0,29093.0,132973.0,-0.54,16.95,66.25,94.43,201.0,0.22,1.47,48.14,XR,NTT,DURABLES
95,WIPRO,243.46,420.00,-2.86,M-LC,89.58,166005.0,15060.0,94390.0,0.00,9.98,56.86,72.51,71.0,0.16,1.21,14.35,XR,NTT,IT
45,INDIAMART,2327.09,4911.36,-48.26,H-SC,67.71,134546.0,11210.0,125760.0,0.01,9.09,93.47,111.05,123.0,0.09,0.98,31.83,AR,ATH,MISC
54,KPIGREEN,497.21,731.64,25.10,H-SC,32.29,128029.0,2732.0,56346.0,-0.95,2.18,44.01,47.15,143.0,0.05,0.94,60.17,MH,ATH,POWER


In [18]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
16,BLUSPRING,226.45,85.1,NaN,NaN,48.96,18637.0,-30956.0,0.0,4.20,-62.42,0.00,-62.42,270.0,-inf,0.14,24.98,XY24,ATH,MISC
24,COFFEEDAY,59.14,80.0,-57.75,L-SC,39.58,63110.0,-50439.0,90487.0,-3.55,-44.42,143.38,35.27,265.0,-0.56,0.46,54.46,XR,NTT,HOTELS
32,ENRIN,1377.95,3087.3,NaN,NaN,95.83,77182.0,42733.0,0.0,2.91,124.05,0.00,124.05,274.0,inf,0.56,18.03,AR,ATH,ELECTRICAL
58,MASFIN,326.60,399.5,-17.80,H-SC,73.44,93555.0,-4425.0,26298.0,0.87,-4.52,28.11,22.32,145.0,-0.17,0.68,35.69,XR,ATH,FINANCE
9,BAJAJFINSV,1994.80,2758.0,-1.53,X-LC,40.62,98304.0,559.0,36835.0,-0.01,0.57,37.47,38.26,18.0,0.02,0.72,31.26,X40,BTT,FINANCE


In [19]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
52,JPPOWER,18.73,26.20,-13.49,L-SC,100.00,174179.0,32337.0,24228.0,1.55,22.80,13.91,39.88,262.0,1.33,1.27,76.52,XY24,NTT,POWER
3,ALKYLAMINE,2347.98,4546.37,8.78,X-SC,98.96,101183.0,220.0,94313.0,-1.87,0.22,93.21,93.63,67.0,0.00,0.74,50.71,SR,ATH,CHEMICALS
92,VIPIND,488.80,489.00,-212.26,H-SC,97.92,80964.0,-14352.0,14387.0,0.10,-15.06,17.77,0.04,157.0,-1.00,0.59,59.69,OX40N,NTT,MISC
15,BLUESTARCO,1646.70,2337.55,-3.59,H-SC,96.88,180800.0,16130.0,52956.0,-1.02,9.80,29.29,41.95,124.0,0.30,1.32,17.99,X40N,ATH,AC
32,ENRIN,1377.95,3087.30,NaN,NaN,95.83,77182.0,42733.0,0.0,2.91,124.05,0.00,124.05,274.0,inf,0.56,18.03,AR,ATH,ELECTRICAL


In [20]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,18.92
1,25,41.88
2,50,70.58


In [21]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.46
LC    33.96
MC    21.50
Name: CurrAlloc%, dtype: float64

In [22]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.96
X40      12.93
XY25     12.04
X40N     11.93
XR       10.01
OX40N     8.19
AR        8.17
MH        1.67
X5K       1.45
X200      1.42
SR        1.22
Name: CurrAlloc%, dtype: float64

In [23]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    20.48
M-SC    15.89
H-LC    14.11
X-LC    12.95
H-MC     9.49
X-MC     7.59
M-LC     5.86
X-SC     4.44
M-MC     4.06
L-SC     2.65
L-LC     1.04
L-MC     0.36
Name: CurrAlloc%, dtype: float64

In [24]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.24,-0.46,34.11
IT,12.08,-9.95,63.94
FINANCE,8.51,-11.96,52.58
BANKS,7.71,-8.14,52.27
MISC,5.85,-20.34,76.75
PAINTS,5.69,-12.30,34.43
ELECTRICAL,5.48,1.60,44.73
HEALTHCARE,4.64,-1.66,31.36
AUTO,4.30,-14.90,65.43


In [25]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2657001.0,26
XR,1089168.0,13
AR,1030340.0,10
X40,689943.0,10
X40N,534356.0,11
XY25,472292.0,8
OX40N,309408.0,11
SR,176687.0,2
MH,87501.0,2


In [26]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,1987439.0,20
M-SC,1768038.0,21
H-LC,613024.0,12
X-LC,540901.0,8
H-MC,535562.0,8
X-MC,417298.0,6
M-MC,414968.0,3
X-SC,310726.0,5
M-LC,258500.0,4


In [27]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      888098.0      6
M-SC       XY24      808140.0      7
H-SC       AR        529210.0      3
M-SC       XR        338136.0      4
M-MC       XY24      303073.0      2
H-MC       XY24      300994.0      3
X-LC       X40       278089.0      3
H-SC       XR        216285.0      3
           X40N      210413.0      4
H-LC       X40N      196381.0      4
M-SC       AR        184052.0      2
           OX40N     163991.0      5
X-MC       X40       157460.0      2
H-LC       X40       156574.0      3
L-SC       XR        152409.0      2
M-LC       XY24      149349.0      2
X-SC       XR        122460.0      1
M-SC       XY25      121374.0      1
X-LC       AR        116296.0      1
M-MC       XR        111895.0      1
X-MC       XY25      103982.0      1
X-LC       XY25       99912.0      3
M-LC       XR         94390.0      1
X-SC       SR         94313.0      1
H-SC       OX40N      87087.0      3
X-MC       XY24       86418.0      1
H-MC       XY25       85001.0      1
M-SC       SR         82374.0      1
H-LC       AR         80816.0      1
           X5K        70302.0      2
M-SC       X40        69971.0      1
H-LC       X200       66679.0      1
H-MC       AR         61949.0      1
L-SC       AR         58017.0      1
H-SC       MH         56346.0      1
L-MC       XR         53593.0      1
X-SC       XY24       52717.0      1
X-MC       X40N       52344.0      1
L-LC       XY25       47262.0      1
X-LC       X40N       46604.0      1
H-LC       XY24       42272.0      1
X-SC       OX40N      41236.0      2
H-MC       MH         31155.0      1
           X40N       28614.0      1
           X40        27849.0      1
L-SC       XY24       24228.0      1
X-MC       OX40N      17094.0      1
M-LC       XY25       14761.0      1

In [28]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 67.0 seconds
